In [1]:
%clear
%reset


Once deleted, variables cannot be recovered. Proceed (y/[n])? n
Nothing done.


In [2]:
import numpy as np
import tensorflow as tf
import datetime
# this code is used to check whether my gradient calculation can be used
import os
import sys
BASE_DIR = os.getcwd()
from tensorflow import keras
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense,Dropout,Conv2D,MaxPooling2D,Flatten
from tensorflow.keras.datasets import mnist
from tensorflow.keras.regularizers import l1
from tensorflow.keras.optimizers import Adam

print('current working dirctory = ',BASE_DIR)
import copy

current working dirctory =  /home/chaotang/文档/Try


In [3]:
LOG_DIR = BASE_DIR + '/log'
# 设定了一个log 的路径，方便如何存入和读取模型
if not os.path.exists(LOG_DIR): os.mkdir(LOG_DIR)
print('current log dirctory = ',LOG_DIR)

current log dirctory =  /home/chaotang/文档/Try/log


# digit recognition Network

In [4]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()
#x_train = np.round(x_train/255)
#x_test = np.round(x_test/255)
x_train = np.round(x_train)/255
x_test = np.round(x_test)/255

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')

x_train = np.reshape(x_train,[60000,28,28,1])
x_test = np.reshape(x_test,[10000,28,28,1])

print(x_train.shape)
print(x_test.shape)
print(y_train.shape)
print(y_test.shape)

(60000, 28, 28, 1)
(10000, 28, 28, 1)
(60000,)
(10000,)


In [5]:
# 传统的定义方法
def digitNet(x,label):
    conv1 = Conv2D(32,(5,5),activation = 'relu',name = 'conv1')(x)
    #print('conv1',conv1.shape) #(20, 26, 26, 32)
    MP1 = MaxPooling2D(pool_size=(2, 2),name = 'MP1')(conv1)
    #print('MP1',MP1.shape) # (20, 13, 13, 32)
    conv2 = Conv2D(32,(3,3),activation = 'relu',name = 'conv2')(MP1)
    #print('conv2',conv2.shape) # (20, 11, 11, 32)
    MP2 = MaxPooling2D(pool_size=(2, 2),name = 'MP2')(conv2)
    #print('MP2',MP2.shape) # (20, 5, 5, 32)
    # Dp1 =Dropout(rate = 0.2,name = 'Dp1')(MP1)
    # go down one more. Add one more conv and one more mp
    # leave the dp1
    Flat = Flatten()(MP2)
    #print('Flat',Flat.shape) # (20, 800)
    fc1 = Dense(128,activation = 'relu',name = 'fc1')(Flat)
    #out = Dense(10,activation = 'softmax',name = 'out')(fc1)
    #print('fc1',fc1.shape) # (20, 128)
    #Dp1 =Dropout(rate = 0.3,name = 'Dp1')(fc1)
    out = Dense(10,name = 'out')(fc1)
    
    loss = tf.nn.sparse_softmax_cross_entropy_with_logits(logits=out, labels=label)
    return loss, out

# Training

In [6]:
# difine two placeholder。This is the old fashion tensorflow method
# 训练数字识别器，loss function选择了和POINTNER++一样的方式去构建 
tf.reset_default_graph()
batchsize = 32
range_val = int(60000/batchsize)

x_pl = tf.placeholder(tf.float32,shape = [batchsize,28,28,1],name = 'x_pl')
label_pl = tf.placeholder(tf.int32,shape = [batchsize],name = 'label_pl')
print(x_pl.shape)
print(label_pl.shape)

loss,output = digitNet(x_pl,label_pl)
optimizer = tf.train.AdamOptimizer()
train_op = optimizer.minimize(loss)

# add summary writers
saver = tf.train.Saver()
merged = tf.summary.merge_all()

for var in tf.trainable_variables():
    print(var.name)
with tf.Session() as sess:
    init = tf.global_variables_initializer()
    sess.run(init)
    train_writer = tf.summary.FileWriter(LOG_DIR, sess.graph)
    for epoch in range(10):
        if(epoch % 1 == 0):
            print('this is the ',epoch,' epoch')
        for i in range(range_val):
            batch_index = i
            train_i = x_train[batchsize * batch_index:batchsize * (batch_index + 1),:]
            label_i = y_train[batchsize * batch_index:batchsize * (batch_index + 1)]
            feed = {x_pl:train_i,label_pl:label_i} 
            my_loss ,_ = sess.run([loss,train_op],feed_dict =feed)
            
    # save the model
    save_path = saver.save(sess,os.path.join(LOG_DIR,'MydigitNet.ckpt'))
    # check the output of the result
    test_img = np.zeros((20,28,28,1))
    test_img[0] = x_test[0].reshape(1,28,28,1)
    networkoutput = sess.run(output,feed_dict = {x_pl:test_img})
    prediction = np.argmax(networkoutput)
    print('networkoutput',networkoutput.shape)
    print('prediction = ',prediction)
    print('label = ',y_test[0])
    #output_img = networkoutput.reshape((28,28))
    #output_img = (output_img * 255).astype('uint8')
    #plt.imshow(output_img)

(32, 28, 28, 1)
(32,)
Instructions for updating:
Colocations handled automatically by placer.
conv1/kernel:0
conv1/bias:0
conv2/kernel:0
conv2/bias:0
fc1/kernel:0
fc1/bias:0
out/kernel:0
out/bias:0
this is the  0  epoch


UnknownError: Failed to get convolution algorithm. This is probably because cuDNN failed to initialize, so try looking to see if a warning log message was printed above.
	 [[node conv1/Conv2D (defined at <ipython-input-5-51e8495a51b9>:3) ]]
	 [[node SparseSoftmaxCrossEntropyWithLogits/SparseSoftmaxCrossEntropyWithLogits (defined at <ipython-input-5-51e8495a51b9>:22) ]]

Caused by op 'conv1/Conv2D', defined at:
  File "/home/chaotang/anaconda3/lib/python3.7/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/home/chaotang/anaconda3/lib/python3.7/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/home/chaotang/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/home/chaotang/anaconda3/lib/python3.7/site-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/home/chaotang/anaconda3/lib/python3.7/site-packages/ipykernel/kernelapp.py",

# Add Noise

In [ ]:
#print('networkoutput = ',networkoutput)
test_index = 3
test_img = np.zeros((20,28,28,1))

test_img[0] = x_test[test_index].reshape(1,28,28,1)
test_label = y_test[test_index].astype('int32')
print('test_label',test_label)
# add Noise

Noise_img = np.zeros((test_img.shape))
Noise_img = test_img + 0.0* np.random.normal(size = (1,28,28,1))
Noise_img = np.clip(Noise_img,0,1)

print('test_img',test_img.shape)
print('Noise_img',Noise_img.shape)
plt.figure()
plt.imshow(test_img[0].reshape((28,28)))
plt.figure()
plt.imshow(Noise_img[0].reshape((28,28)))

# Varify different position of pixel change would have different effect on score

In [ ]:
# Create a Flat image
Flat_img = np.zeros((1,28,28,1))
# Flat_img
# Test the sensitivity of the pixel to the current score
score_mat = np.zeros((11,28,28,1))

for i in range(11):
    score_mat[i,5,4,:] = 0.1 * i

# 证明不同位置变化对分数有影响 

In [ ]:
# restore the model
# also remember to reset_everything to defaut!!!
tf.reset_default_graph()
MODEL_PATH = os.path.join(LOG_DIR,'MydigitNet.ckpt')
# Have to redifine Graph
x_pl = tf.placeholder(tf.float32,shape = [None,28,28,1],name = 'x_pl')
label_pl = tf.placeholder(tf.int32,shape = [None],name = 'label_pl')

loss,output = digitNet(x_pl,label_pl)

saver = tf.train.Saver()
with tf.Session() as sess: 
    saver.restore(sess, MODEL_PATH)
    print("Model restored.")
    #############################################################
    ### Expect to see score to be all zero when Flat_img input###
    #############################################################
    Flat_feed = {x_pl:Flat_img}
    FlatOut= sess.run(output,feed_dict =Flat_feed) # Flat input should output 0 score for all the class
    Net_offset = FlatOut[0,...]
    print('Net_offset = ',Net_offset)
    #
    # The output socre for an important location
    feed = {x_pl:score_mat}
    output1= sess.run(output,feed_dict =feed)
    S = np.zeros((10,11))
    for i in range(10):
        S[i,:] = output1[:,i] - Net_offset[i]
        plt.figure()
        plt.plot(S[i])
        plt.title(i)
    #for i in range(11):
    #    print('Output Score = ',output1[i,0])

# Get the Weight Mask


In [ ]:
Mask_w = np.zeros((28,28,10)) # (28,28) across 10 classes

In [ ]:
# restore the model
# also remember to reset_everything to defaut!!!
tf.reset_default_graph()
MODEL_PATH = os.path.join(LOG_DIR,'MydigitNet.ckpt')
# Have to redifine Graph
x_pl = tf.placeholder(tf.float32,shape = [None,28,28,1],name = 'x_pl')
label_pl = tf.placeholder(tf.int32,shape = [None],name = 'label_pl')

loss,output = digitNet(x_pl,label_pl)

saver = tf.train.Saver()
with tf.Session() as sess: 
    saver.restore(sess, MODEL_PATH)
    print("Model restored.")
    #############################################################
    ### Expect to see score to be all zero when Flat_img input###
    #############################################################
    Flat_feed = {x_pl:Flat_img} ## 但是这个offset 是全局的offset，真的能够适用吗?
    FlatOut= sess.run(output,feed_dict =Flat_feed)
    Net_offset = FlatOut[0,...]
    print('Net_offset = ',Net_offset)
    
    # The output socre for all important location
    count = 0
    for row in range (28):
        for col in range(28): 
            score_mat_i = np.zeros((11,28,28,1))
            for i in range(11):
                score_mat_i[i,row,col,:] = 0.1 * i
                
            feed = {x_pl:score_mat_i}
            output_cpoint= sess.run(output,feed_dict =feed)
            S = np.zeros((10,11))
            for i in range(10):
                S[i,:] = output_cpoint[:,i] - Net_offset[i]
            # S[i,:] 是第i个class 的分数
            S_min = np.amin(S,axis = 1)# shape = (10,)
            S_max = np.amax(S,axis = 1)
            # Two ways to define the sensitivity
            # Sensitivity = S_max - S_min # 这个似乎不太合理，会把副方向的激发也当成正方向的激发
            # Sensitivity = S_max - S[:,9]
            Sensitivity =  S[:,9]
            Mask_w[row,col,:] = Sensitivity

In [ ]:
for i in range(1):
    plt.figure()
    imask = np.clip(Mask_w[...,i],0,1)
    #imask = Mask_w[...,i]/np.amax(imask)
    print(imask,np.amax(Mask_w[...,i]))
    #plt.imshow(Mask_w[...,i]/np.amax(Mask_w[...,i]))
    plt.imshow(imask)
    plt.title(i)


# Denoise Step

In [ ]:
# 上面图过后成功完成训练了，接下来就是保存模型并提取模型，然后用自编码模型完成老师的任务了
# restore the model
# also remember to reset_everything to defaut!!!
tf.reset_default_graph()

# 下文的NoiseOutput是上一个restore的输出结果，等价于Noiseoutput = digitNet(x_pl,label_pl)

batchsize = 20
MODEL_PATH = os.path.join(LOG_DIR,'MydigitNet.ckpt')
###############################
### Have to redifine Graph ####
###############################
# this is the new graph
x_pl = tf.placeholder(tf.float32,shape = [batchsize,28,28,1],name = 'x_pl')
# x_pl = Noise_img
label_pl = tf.placeholder(tf.int32,shape = [batchsize],name = 'label_pl')

deltax = tf.get_variable('deltax',dtype=tf.float32,shape = x_pl.shape,initializer = tf.zeros_initializer)
# 必须要固定shape 才行，否则不允许如此设定变量

New_x = tf.clip_by_value((x_pl + deltax),0,1)# this may need a clip operation
#New_x = x_pl + deltax

loss,output = digitNet(New_x,label_pl)
pre_trained_list = []
var_list = []
for var in tf.trainable_variables():
    print(var.name)
    var_list.append(var)
pre_trained_list = var_list[1:]
print('###############################')
for var in pre_trained_list:
    print(var.name)
print('This is the train-graph name')
print('###############################')

saver = tf.train.Saver(var_list=pre_trained_list)

##################################
####### Graph has been redefine###
#################################
# 训练条件配置
train_var = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES,scope = 'deltax:0')
displacement_mat = np.zeros((500,28,28,1)) # store the displacement for every 100 iteration
##############################################################
with tf.Session() as sess:
    sess.run(tf.variables_initializer([deltax]))
    saver.restore(sess,MODEL_PATH)
    print("Model restored.")
    #print('NoiseOutput',NoiseOutput[0])
    print('TestLabel',test_label)
    # define loss function (energy) 
    # E1 = deltaX^2
    E1 = 0.5 * tf.reduce_sum(tf.square(deltax[0])) # this become a scalar/// the shape of deltax[0] = (28,28,1)
    # E2 = Si ^ 2
    E2 = -output[0,test_label]
    
    energy_total = E1 + E2
    # optimizer define
    optimizer = tf.train.GradientDescentOptimizer(0.005)
    train_op = optimizer.minimize(energy_total,var_list = train_var)
    feed = {x_pl:Noise_img[0].reshape((1,28,28,1))}
    print('Noise_img',Noise_img.shape)
    loss_mat = np.zeros((4000,))
    for i in range(4000):
        _,loss_in_loop = sess.run([train_op,energy_total],feed_dict = feed)
        #print('energy_total',loss_in_loop)

        loss_mat[i] = loss_in_loop
        #if i%1 ==0:
           # j = int(i/1)
            #displacement_mat[j,...] = deltax[0].eval()  
        
    print('deltax= ',deltax[0].eval().shape)
    displacement = deltax[0].eval()
    #print('displacement = ',displacement)

In [ ]:
plt.plot(loss_mat)
print(np.amin(loss_mat))

In [ ]:
# restore the model
# also remember to reset_everything to defaut!!!
tf.reset_default_graph()
print('displacement_mat',displacement_mat.shape)

MODEL_PATH = os.path.join(LOG_DIR,'MydigitNet.ckpt')
# Have to redifine Graph
x_pl = tf.placeholder(tf.float32,shape = [None,28,28,1],name = 'x_pl')
label_pl = tf.placeholder(tf.int32,shape = [None],name = 'label_pl')

loss,output = digitNet(x_pl,label_pl)

saver = tf.train.Saver()
print('test_label = ',test_label)
Noise_mat = np.zeros((500,28,28,1))
for i in range(500):
    Noise_mat[i,...] = Noise_img[0,...]
with tf.Session() as sess: 
    saver.restore(sess, MODEL_PATH)
    print("Model restored.")
    # 设置输出小数点位数
    np.set_printoptions(precision=10)
    
    feed1 = {x_pl:test_img}
    output1 = sess.run(output,feed_dict =feed1)
    
    #Noise_img[:,16,9,:] +=1
    
    feed2 = {x_pl:Noise_img}
    output2 = sess.run(output,feed_dict =feed2)
    
    feed3 = {x_pl:Noise_img + displacement.reshape(1,28,28,1)}
    output3 = sess.run(output,feed_dict =feed3)
    
    feed4 = {x_pl:Noise_mat[0:20,...] + displacement_mat[0:20,...]}
    output4 = sess.run(output,feed_dict =feed4)
    
    print('done with image output,calculating Error...')
   

In [ ]:
# 设置输出小数点位数
np.set_printoptions(precision=5)
plt.imshow(test_img[0].reshape((28,28)))
print('Original score for testLabel= ',output1[0,test_label]) #test_label
print('Original output = ',output1[0,:]) #test_label

plt.figure()
Old_noise = np.clip(Noise_img[0],0,1).reshape((28,28))
plt.imshow(Old_noise)
print('Noise score for testLabel= ',output2[0,test_label])
print('Noise output = ',output2[0,:]) #test_label

plt.figure()
# Change 
Processed = np.clip(Noise_img[0] + displacement,0,1).reshape((28,28))
plt.imshow(Processed)
print('New score for testLabel= ',output3[0,test_label])
print('New output= ',output3[0,:])

print('different = ',  np.square(Processed-Old_noise).mean())
print('Processed',Processed[5,4])
#plt.figure()
#plt.imshow(Mask_w[...,test_label]/np.amax(Mask_w[...,test_label]))

# compute the classification error

In [ ]:
#################################################################
########### Then compute the classification error################
#################################################################
feed5 = {x_pl:x_train}
trainset_out = sess.run(output,feed_dict =feed5)
#print('trainset_out',trainset_out.shape)
train_pred = np.argmax(trainset_out,1)
#print('train_pred',train_pred.shape)
correct_train = 0
for i in range(train_pred.shape[0]):
    correct_train += int(train_pred[i]==y_train[i])
print('The accuracy of classification in training set',correct_train/train_pred.shape[0])
print('With correct case',correct_train)

feed6 = {x_pl:x_test}
testset_out = sess.run(output,feed_dict =feed6)
test_pred = np.argmax(testset_out,1)
correct_test = 0
for i in range(test_pred.shape[0]):
    correct_test += int(test_pred[i]==y_test[i])
print('The accuracy of classification in test set',correct_test/test_pred.shape[0])
print('With correct case',correct_test)

# Implement the Mask to GradientDecent

In [ ]:
# restore the model
# also remember to reset_everything to defaut!!!
tf.reset_default_graph()
batchsize = 20
MODEL_PATH = os.path.join(LOG_DIR,'MydigitNet.ckpt')
######################################################################
### Have to redifine Graph ####
###############################
# this is the new graph
x_pl = tf.placeholder(tf.float32,shape = [batchsize,28,28,1],name = 'x_pl')
# x_pl = Noise_img
label_pl = tf.placeholder(tf.int32,shape = [batchsize],name = 'label_pl')

deltax = tf.get_variable('deltax',dtype=tf.float32,shape = x_pl.shape)
# 必须要固定shape 才行，否则不允许如此设定变量

New_x = tf.clip_by_value((x_pl + deltax),0,1)# this may need a clip operation
#New_x = x_pl + deltax

loss,output = digitNet(New_x,label_pl)
pre_trained_list = []
var_list = []
for var in tf.trainable_variables():
    #print(var.name)
    var_list.append(var)
pre_trained_list = var_list[1:]
print('###############################')
#for var in pre_trained_list:
    #print(var.name)
print('This is the train-graph name')
print('###############################')

saver = tf.train.Saver(var_list=pre_trained_list)
##################################
####### Graph has been redefine###
##################################################################################
# 训练条件配置
train_var = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES,scope = 'deltax:0')
with tf.Session() as sess:
    sess.run(tf.variables_initializer([deltax]))
    saver.restore(sess,MODEL_PATH)
    print("Model restored.")
    
    # define loss function (energy) 
    # E1 = deltaX^2
    E1 =  0.5 * tf.reduce_sum(tf.square(deltax[0])) # this become a scalar/// the shape of deltax[0] = (28,28,1)
    # E2 = Si 
    E2 = -output[0,test_label]
    
    energy_total = E1 + E2
    # optimizer define
    optimizer = tf.train.GradientDescentOptimizer(0.0002)
    train_op = optimizer.minimize(energy_total,var_list = train_var)
    feed = {x_pl:Noise_img}
    print('Noise_img',Noise_img.shape)
    for i in range(5000):
        _,loss_in_loop = sess.run([train_op,energy_total],feed_dict = feed)
        #print('energy_total',loss_in_loop)
      
        #if i%1 ==0:
           # j = int(i/1)
            #displacement_mat[j,...] = deltax[0].eval()  
        
    print('deltax= ',deltax[0].eval().shape)
    displacement = deltax[0].eval()